# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [173]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import os 

In [174]:
data = pd.read_csv(os.getcwd()+'/customer_product_sales.csv')

In [175]:
print(data.shape)
data.head()

(68584, 7)


,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [176]:
tq = data.pivot_table(index=('CustomerID','ProductName'), values='Quantity')#, values='ProductName',aggfunc=({'ProductName':len}))
tq.head(15)

Quantity
CustomerID ProductName                              
33         Apricots - Dried                        1
           Assorted Desserts                       1
           Bandage - Flexible Neon                 1
           Bar Mix - Pina Colada, 355 Ml           1
           Beans - Kidney, Canned                  1
           Beef - Chuck, Boneless                  1
           Beef - Prime Rib Aaa                    1
           Beer - Original Organic Lager           1
           Beer - Rickards Red                     1
           Black Currants                          1
           Blackberries                            1
           Bread - Italian Roll With Herbs         1
           Bread Foccacia Whole                    1
           Cake - Box Window 10x10x2.5             1
           Cassis                                  1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [177]:
pc = data.pivot_table(index=('ProductName'),columns='CustomerID', values='Quantity').fillna(0)
pc

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,25.0
Apricots Fresh,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
Arizona - Green Tea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0
Artichokes - Jerusalem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [327]:
from scipy.spatial.distance import pdist, squareform

matrix = pd.DataFrame(1/(1+squareform(pdist(pc.T, 'euclidean'))),index=pc.columns,columns=pc.columns)
matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
464,0.089695,0.083020,0.087047,0.086688,0.085638,1.000000,0.085638,0.092610,0.087779,0.087047,...,0.004814,0.005121,0.004993,0.005445,0.004920,0.005042,0.004877,0.005039,0.004556,0.004897
477,0.085297,0.084959,0.085638,0.085983,0.085638,0.085638,1.000000,0.090909,0.086333,0.087047,...,0.004816,0.005124,0.005005,0.005436,0.004928,0.005042,0.004882,0.005039,0.004556,0.004885
639,0.088913,0.083651,0.086333,0.091325,0.089301,0.092610,0.090909,1.000000,0.093498,0.091747,...,0.004814,0.005122,0.004993,0.005441,0.004917,0.005039,0.004880,0.005033,0.004563,0.004883
649,0.088152,0.085638,0.087047,0.085983,0.084959,0.087779,0.086333,0.093498,1.000000,0.088530,...,0.004811,0.005115,0.004999,0.005437,0.004923,0.005039,0.004886,0.005045,0.004558,0.004877


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [326]:
filt = matrix[33][1:].sort_values(ascending=False).head(5)
filt

CustomerID
1577    0.536232
264     0.535000
3531    0.535000
3909    0.533679
756     0.533333
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [196]:
tq2 = tq.reset_index(level=['CustomerID','ProductName'])

In [242]:
select = tq2[tq2['CustomerID'].isin(filt.index)]

In [243]:
select.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [324]:
ranked=select.groupby('ProductName').agg({'Quantity':sum}).sort_values('Quantity', ascending=False)
ranked.head()

,Quantity
ProductName,
"Soup - Canadian Pea, Dry Mix",68
Pastry - Butterscotch Baked,51
Ezy Change Mophandle,51
Salmon - Sockeye Raw,51
"Lemonade - Natural, 591 Ml",51


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [322]:
merge.head()

,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,51
Apricots Fresh,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,17
Artichokes - Jerusalem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17
Bacardi Breezer - Tropical,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,17


In [276]:
merge = pd.merge(pc,ranked,on='ProductName')
missing = merge[(merge[33]==0)&(merge['Quantity']>0)]
missing.sort_values('Quantity',ascending=False).index[:5]

Index(['Pecan Raisin - Tarts', 'Beets - Candy Cane, Organic',
       'Kellogs Special K Cereal', 'Veal - Inside', 'Juice - Lime'],
      dtype='object', name='ProductName')

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [285]:
recomendations = {}
CustomerIDs = set(data['CustomerID'])

In [286]:
for customers in CustomerIDs:
    filt = matrix[customers][1:].sort_values(ascending=False).head(5)
    select = tq2[tq2['CustomerID'].isin(filt.index)]
    ranked=select.groupby('ProductName').agg({'Quantity':sum}).sort_values('Quantity', ascending=False)
    merge = pd.merge(pc,ranked,on='ProductName')
    missing = merge[(merge[customers]==0)&(merge['Quantity']>0)]
    rec = [i for i in missing.sort_values('Quantity',ascending=False).index[:5]]
    recomendations.update({customers: rec})

In [287]:
recomendations

{83973: ['Pepper - Paprika, Hungarian',
  'Lamb - Pieces, Diced',
  'Soup - Campbells, Lentil',
  'Vaccum Bag 10x13',
  'Crackers - Trio'],
 59399: ['Nut - Pistachio, Shelled',
  'Lettuce - California Mix',
  'Flavouring - Orange',
  'Foam Dinner Plate',
  'Cookie Chocolate Chip With'],
 92168: ['Bandage - Flexible Neon',
  'Bread - Multigrain',
  'Vol Au Vents',
  'Lettuce - Spring Mix',
  'Wine - Vineland Estate Semi - Dry'],
 49159: ['Vinegar - Sherry',
  'Doilies - 5, Paper',
  'Water, Tap',
  'Tray - 16in Rnd Blk',
  'Spinach - Baby'],
 18441: ['Veal - Inside',
  'Placemat - Scallop, White',
  'Initation Crab Meat',
  'Nantuket Peach Orange',
  'Cheese - Taleggio D.o.p.'],
 22536: ['Bread - Italian Roll With Herbs',
  'Pail For Lid 1537',
  'Cheese Cloth No 100',
  'Lettuce - Treviso',
  'Rabbit - Whole'],
 86028: ['Soupcontfoam16oz 116con',
  'Thermometer Digital',
  'Pork - Kidney',
  'Peas - Pigeon, Dry',
  'Soup Campbells - Italian Wedding'],
 75791: ['Sage - Ground',
  'Turke

In [294]:
recomendations.values()

dict_values([['Pepper - Paprika, Hungarian', 'Lamb - Pieces, Diced', 'Soup - Campbells, Lentil', 'Vaccum Bag 10x13', 'Crackers - Trio'], ['Nut - Pistachio, Shelled', 'Lettuce - California Mix', 'Flavouring - Orange', 'Foam Dinner Plate', 'Cookie Chocolate Chip With'], ['Bandage - Flexible Neon', 'Bread - Multigrain', 'Vol Au Vents', 'Lettuce - Spring Mix', 'Wine - Vineland Estate Semi - Dry'], ['Vinegar - Sherry', 'Doilies - 5, Paper', 'Water, Tap', 'Tray - 16in Rnd Blk', 'Spinach - Baby'], ['Veal - Inside', 'Placemat - Scallop, White', 'Initation Crab Meat', 'Nantuket Peach Orange', 'Cheese - Taleggio D.o.p.'], ['Bread - Italian Roll With Herbs', 'Pail For Lid 1537', 'Cheese Cloth No 100', 'Lettuce - Treviso', 'Rabbit - Whole'], ['Soupcontfoam16oz 116con', 'Thermometer Digital', 'Pork - Kidney', 'Peas - Pigeon, Dry', 'Soup Campbells - Italian Wedding'], ['Sage - Ground', 'Turkey - Oven Roast Breast', 'Zucchini - Yellow', 'Lime Cordial - Roses', 'Bananas'], ['Guinea Fowl', 'Garlic - El

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [301]:
rec_df = pd.DataFrame({'CustomerID':recomendations.keys(),
                      'Recomendation_1':recomendations.values()})

In [317]:
rec_df1 = pd.DataFrame(recomendations).T.rename(columns={0:'Rec_1',1:'Rec_2',2:'Rec_3',
                                                         3:'Rec_4',4:'Rec_5'})

In [318]:
rec_df1.head()

,Rec_1,Rec_2,Rec_3,Rec_4,Rec_5
83973,"Pepper - Paprika, Hungarian","Lamb - Pieces, Diced","Soup - Campbells, Lentil",Vaccum Bag 10x13,Crackers - Trio
59399,"Nut - Pistachio, Shelled",Lettuce - California Mix,Flavouring - Orange,Foam Dinner Plate,Cookie Chocolate Chip With
92168,Bandage - Flexible Neon,Bread - Multigrain,Vol Au Vents,Lettuce - Spring Mix,Wine - Vineland Estate Semi - Dry
49159,Vinegar - Sherry,"Doilies - 5, Paper","Water, Tap",Tray - 16in Rnd Blk,Spinach - Baby
18441,Veal - Inside,"Placemat - Scallop, White",Initation Crab Meat,Nantuket Peach Orange,Cheese - Taleggio D.o.p.


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [319]:
recomendations = {}
CustomerIDs = set(data['CustomerID'])

In [320]:
matrix = pd.DataFrame(1/(1+squareform(pdist(pc.T, 'jaccard'))),index=pc.columns,columns=pc.columns)

for customers in CustomerIDs:
    filt = matrix[customers][1:].sort_values(ascending=False).head(5)
    select = tq2[tq2['CustomerID'].isin(filt.index)]
    ranked=select.groupby('ProductName').agg({'Quantity':sum}).sort_values('Quantity', ascending=False)
    merge = pd.merge(pc,ranked,on='ProductName')
    missing = merge[(merge[customers]==0)&(merge['Quantity']>0)]
    rec = [i for i in missing.sort_values('Quantity',ascending=False).index[:5]]
    recomendations.update({customers: rec})

In [321]:
rec_df = pd.DataFrame({'CustomerID':recomendations.keys(),
                      'Recomendation_1':recomendations.values()})
rec_df1 = pd.DataFrame(recomendations).T.rename(columns={0:'Rec_1',1:'Rec_2',2:'Rec_3',
                                                         3:'Rec_4',4:'Rec_5'})
rec_df1.head()

,Rec_1,Rec_2,Rec_3,Rec_4,Rec_5
83973,Kellogs All Bran Bars,Soup Knorr Chili With Beans,Pork - Belly Fresh,Muffin - Zero Transfat,Cake - Box Window 10x10x2.5
59399,Vanilla Beans,Kellogs All Bran Bars,Appetizer - Sausage Rolls,Muffin - Carrot Individual Wrap,Cheese - Parmesan Cubes
92168,"Soup - Canadian Pea, Dry Mix",Olives - Stuffed,Wiberg Super Cure,Crackers Cheez It,Tea - English Breakfast
49159,Sponge Cake Mix - Chocolate,"Wine - White, Schroder And Schyl",Pastry - Cheese Baked Scones,Flavouring - Orange,Soup Campbells - Italian Wedding
18441,"Beef - Ground, Extra Lean, Fresh",Broom - Corn,Macaroons - Two Bite Choc,Chinese Foods - Chicken,Sword Pick Asst
